<a href="https://colab.research.google.com/github/deltorobarba/astrophysics/blob/main/ground_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ground State Calculations in Quantum Chemistry**

**Variational Quantum Eigensolve (VQE) for Hydrogen Molecule (H₂)**

In [ ]:
!pip install pennylane -q

In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define the molecular system
symbols = ['H', 'H']
coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.7414])  # Atomic units

# Generate the molecular Hamiltonian directly in PennyLane
H, qubits = qml.qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=0,
    mult=1,  # Singlet state
    basis='sto-3g',
    active_electrons=2,
    active_orbitals=2,
    mapping='jordan_wigner'
)

# Define the quantum device
dev = qml.device('default.qubit', wires=qubits)

# Define the ansatz (parameterized quantum circuit)
def ansatz(params):
    for i in range(qubits):
        qml.RY(params[i], wires=i)
    qml.CNOT(wires=[0, 1])

# Define the cost function (expectation value of the Hamiltonian)
@qml.qnode(dev)
def cost_fn(params):
    ansatz(params)
    return qml.expval(H)

# Perform the VQE optimization
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)
params = np.random.randn(qubits)
for i in range(100):
    params = optimizer.step(cost_fn, params)

# Extract the ground state energy
ground_state_energy = cost_fn(params)

# Compute excited state energies (modify the ansatz and cost function accordingly)
# ...

print("Ground state energy:", ground_state_energy)

Ground state energy: -0.8830050342313341


**Approach**
1. Molecular System Definition: Specify the atomic symbols and coordinates (in atomic units) for H₂
2. Hamiltonian Generation: Use PennyLane's qml.qchem.molecular_hamiltonian to directly generate the qubit Hamiltonian, specifying the charge, multiplicity, basis set, active electrons and orbitals, and fermion-to-qubit mapping.
3. Quantum Device, Ansatz, and Cost Function: These remain similar to the previous example, but now we use the H generated by PennyLane directly in the cost function.
4. VQE Optimization: Perform the optimization as before to find the ground state energy.
5. Excited States: Adapt the ansatz and cost function to target excited states using appropriate techniques.

**Considerations**

* active_electrons and active_orbitals: These parameters control the size of the active space, which can significantly impact the accuracy and computational cost. Choose them carefully based on your system and desired accuracy.
* Excited State Calculation: Remember to modify the ansatz and cost function appropriately to target specific excited states. You might need to employ techniques like subspace search or symmetry constraints.

**Next Steps:**
- You can extend this approach to more complex molecules by changing the molecular geometry and basis set.
- You can also use more sophisticated ansätze like Unitary Coupled Cluster (UCC) for more accurate results.
- PennyLane also supports other quantum devices such as IBMQ, which can be integrated for running on real quantum hardware.